# Closest Stations
Lets see if we can match our houses to their nearest station, and figure out how far they are from them!

In [3]:
import pandas as pd
map_base = pd.read_csv('../DATA/RealEstate2.csv')
stats = pd.read_csv('../DATA/StationEntrances2020_v4.csv').\
            groupby('Train_Station').agg({'LAT':'mean', 'LONG':'mean'}).reset_index()

In [6]:
len(map_base), len(stats)

(61, 351)

In [36]:
import geopandas as gpd
from shapely.geometry import Point

gdfm = gpd.GeoDataFrame(map_base, 
                        geometry=[Point(xy) for xy in zip(map_base.lon, map_base.lat)])
gdfs = gpd.GeoDataFrame(stats.drop(['LONG', 'LAT'], axis=1), 
                        geometry=[Point(xy) for xy in zip(stats.LONG, stats.LAT)])

In [37]:
from scipy.spatial import cKDTree

def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

out = ckdnearest(gdfm, gdfs)

In [38]:
also = stats[stats.Train_Station.isin(out.Train_Station.unique())]

In [40]:
# out

In [41]:
map_base['url3'] = [f'<a href = "{x}" target = "_blank">Original Source</a>' for x in map_base['url']]
import numpy as np
map_base['price_group'] = [str(np.round(x/50000)*50)[:-2] for x in map_base.max_price]

In [47]:
import mapfuncs as mf
f, m = mf.create_map(out[out.dist<0.005], 
             lat_col = 'lat',
             lon_col = 'lon', all = True,
             group_col = 'price_group', zoom = 14, disable_cluster = 8,
             strings = ['add', 'bed', 'bath', 'car', 'headline', 'desc', 'url3'],
             sort_col = 'max_price', photo_col = 'photo')

In [56]:
# import folium
# also.apply(lambda row:folium.Marker(location=[row['LAT'], row['LONG']],
#                                     icon = folium.Icon(color='black', 
#                                                                icon_color = "white",
#                                                                prefix = 'fa',
#                                                                icon = 'train')
#                                                ).add_to(m), axis=1)
# f.add_child(m)
# f